# keywordExtraction

In [20]:
! pip install pypdf
! pip install ftfy
! pip install pyspellchecker
! pip install nltk

In [31]:
import re
import nltk
from pypdf import PdfReader
import ftfy
import unicodedata
from spellchecker import SpellChecker
import nltk
nltk.download('stopwords')
import networkx as nx
from nltk.stem import WordNetLemmatizer
from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures
# --- PERSIAPAN ---
try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')
import matplotlib.pyplot as plt



#  --- KONFIGURASI NLTK ---
# Memastikan resource tokenizer tersedia
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    print("[INFO] Mengunduh data NLTK...")
    nltk.download('punkt')
    nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## 1. Load data </br>
1. membaca data pdf </br>
2. menghapus kata kata noise, hapus baris yang terlalu pendek<br>
3. memotong refference/daftar pustaka<br>

In [9]:
def extract_text_from_pdf(pdf_path):
    """
    Membaca PDF, lalu memperbaiki encoding (ftfy) dan normalisasi Unicode.
    """
    try:
        reader = PdfReader(pdf_path)
        full_text = ""
        for page in reader.pages:
            text = page.extract_text()
            if text:
                # 1. FTFY: Perbaiki encoding teks yang rusak (mojibake)
                # Ini memperbaiki karakter aneh sebelum diproses lebih lanjut
                text = ftfy.fix_text(text)

                # 2. Unicode Normalization: Memecah ligatures (ﬁ -> f i)
                text = unicodedata.normalize('NFKD', text)

                full_text += text + "\n"
        return full_text
    except Exception as e:
        print(f"[ERROR] Gagal membaca PDF: {e}")
        return ""

def clean_journal_text(raw_text):
    """
    Membersihkan noise, header/footer, dan memotong bagian referensi.
    """
    if not raw_text:
        return ""

    # 1. Normalisasi Awal
    text = raw_text.strip()

    # --- TAHAP 1: MENGHAPUS NOISE UMUM (REGEX) ---
    # Menghapus citasi angka standar ex: [1], [1, 2], [1-3]
    text = re.sub(r'\[[\d,\s\-]+\]', '', text)
    # Menghapus URL dan DOI
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'doi: \S+', '', text, flags=re.IGNORECASE)
    # Menghapus simbol LaTeX/Matematika ($...$)
    text = re.sub(r'\$.*?\$', '', text)

    # --- TAHAP 2: MENGHAPUS HEADER/FOOTER & METADATA PER BARIS ---
    lines = text.split('\n')
    cleaned_lines = []

    # Daftar pola noise di header/footer (Sesuaikan jika ada pola lain)
    noise_patterns = [
        r'^\d+\s*of\s*\d+$',         # Halaman: "1 of 24"
        r'^\d+$',                    # Halaman: Angka saja "12"
        r'Diagnostics 2024',         # Nama Jurnal/Tahun
        r'Authorized licensed use',  # License info
        r'Downloaded on',            # Timestamp download
        r'Creative Commons',         # Copyright
        r'©',                        # Simbol copyright
        r'IEEE Xplore',
        r'MDPI',
        r'Vol\.\s*\d+',              # Volume info
        r'ISSN\s*\d+',               # ISSN
    ]

    for line in lines:
        line = line.strip()
        is_noise = False

        # Cek pola noise
        for pattern in noise_patterns:
            if re.search(pattern, line, re.IGNORECASE):
                is_noise = True
                break

        # Hapus baris yang terlalu pendek (sisa potongan) kecuali Judul Bab (Huruf Besar)
        if len(line) < 4 and not re.match(r'^[A-Z0-9\.]+$', line):
            is_noise = True

        if not is_noise:
            cleaned_lines.append(line)

    # Gabungkan kembali baris yang sudah bersih
    text = '\n'.join(cleaned_lines)

    # --- TAHAP 3: STRUKTURISASI & PERBAIKAN KATA ---

    # a. Mencari posisi awal (Abstract)
    # Catatan: Jika ingin mengambil Judul paling atas, matikan bagian ini.
    # Namun, biasanya bagian atas penuh dengan nama penulis/afiliasi yang sulit dibersihkan.
    start_match = re.search(r'(Abstract|Abstrak)[-—:.]?', text, re.IGNORECASE)
    if start_match:
        # Ambil teks mulai dari kata "Abstract"
        text = text[start_match.start():]

    # b. Perbaiki kata terpotong (Hyphenation) sebelum tokenisasi
    # Contoh: "process-\ning" menjadi "processing"
    text = re.sub(r'-\n\s*', '', text)

    # c. Jadikan satu baris panjang (Single paragraph flow)
    text = re.sub(r'\s+', ' ', text).strip()

    # --- TAHAP 4: MEMOTONG BAGIAN AKHIR (REFERENCES) MENGGUNAKAN NLTK ---

    # Pecah teks menjadi daftar kalimat
    sentences = nltk.sent_tokenize(text)

    # Pola Regex untuk mendeteksi Header Bagian Akhir
    header_pattern = r'^\s*(?:\d+\.?\s*)?(References|Daftar Pustaka|Bibliography|Acknowledgments|Ucapan Terima Kasih|Funding|Conflicts of Interest|Rujukan)\b'

    cutoff_index = -1

    # Loop setiap kalimat
    for i, sent in enumerate(sentences):
        # Cek kalimat pendek yang mengandung kata kunci header
        if len(sent) < 1000 and re.search(header_pattern, sent, re.IGNORECASE):
            print(f"[LOG] Memotong teks mulai dari: '{sent}'")
            cutoff_index = i
            break # Stop di temuan pertama
        else:
          print("tidak ditemukan")

    # Potong list kalimat
    if cutoff_index != -1:
        final_sentences = sentences[:cutoff_index]
    else:
        final_sentences = sentences

    # Gabungkan kembali
    return ' '.join(final_sentences)

In [10]:
input_pdf = "/content/drive/MyDrive/Semester 7/PPW/UAS_PROJEK/Classifying_Alzheimer_Disease_using_VGG19.pdf"
print(f"Sedang memproses: {input_pdf} ...")
# 1. Ekstrak teks mentah dari PDF
raw_text = extract_text_from_pdf(input_pdf)

if raw_text:
    # 2. Bersihkan teks
    clean_text = clean_journal_text(raw_text)

    # 3. Tampilkan atau Simpan Hasil
    print("\n" + "="*40)
    print("HASIL PREPROCESSING BERSIH")
    print("="*40)
    print(clean_text[:1000] + "...\n(dan seterusnya)") # Print 1000 karakter pertama

    # Opsi: Simpan ke file teks
    output_filename = "hasil_bersih2.txt"
    with open(output_filename, "w", encoding="utf-8") as f:
        f.write(clean_text)
    print(f"\n[SUKSES] Hasil bersih disimpan ke '{output_filename}'")
else:
    print("[GAGAL] Tidak ada teks yang bisa diekstrak.")


Sedang memproses: /content/drive/MyDrive/Semester 7/PPW/UAS_PROJEK/Classifying_Alzheimer_Disease_using_VGG19.pdf ...
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tidak ditemukan
tid

### Cleaning lanjutan

In [12]:
def advanced_preprocessing(text):
    """
    Melakukan pembersihan mendalam khusus untuk keperluan Keyword Extraction.
    Fitur:
    1. Hapus sitasi (Author, Year)
    2. Hapus email/URL
    3. Hapus kalimat kotor (banyak angka/simbol)
    4. Hapus kalimat terlalu pendek
    """

    # --- TAHAP 1: PEMBERSIHAN POLA REGEX (LEVEL TEKS) ---

    # 1. Hapus Email
    text = re.sub(r'\S+@\S+', '', text)

    # 2. Hapus URL (www, http, https)
    text = re.sub(r'http\S+|www\.\S+', '', text)

    # 3. Hapus Sitasi format (Name et al., 2012) atau (Name, 2012)
    # Logika: Cari kurung yang diakhiri dengan 4 digit angka (tahun)
    text = re.sub(r'\([^)]*?\d{4}\)', '', text)

    # 4. Hapus referensi [1], [12] (backup jika tahap sebelumnya lolos)
    text = re.sub(r'\[\d+(?:,\s*\d+)*\]', '', text)

    # 5. Hapus kata-kata sisa referensi umum
    text = re.sub(r'\b(et al\.|Ibid\.|Vol\.|pp\.|Fig\.|Table)\b', '', text, flags=re.IGNORECASE)

    # --- TAHAP 2: FILTERING PER KALIMAT ---

    sentences = nltk.sent_tokenize(text)
    clean_sentences = []

    for sent in sentences:
        # A. Hapus kalimat yang terlalu pendek (Ide Anda)
        # Kita hitung jumlah kata (token)
        words = sent.split()
        if len(words) < 5:
            continue # Skip kalimat ini

        # B. Hapus kalimat "Matematika/Tabel" (Ide Anda)
        # Logika: Jika jumlah angka dan simbol > 30% dari total karakter kalimat, buang.
        num_digits = len(re.findall(r'\d', sent))
        num_symbols = len(re.findall(r'[+\-*/%=<>±]', sent)) # Simbol math
        total_chars = len(sent.replace(" ", ""))

        if total_chars > 0:
            ratio_noise = (num_digits + num_symbols) / total_chars
            if ratio_noise > 0.25: # Jika lebih dari 25% isinya angka/simbol
                continue # Skip kalimat ini (kemungkinan besar tabel atau rumus)

        # C. Pembersihan akhir per kalimat (Spasi ganda & trim)
        sent = re.sub(r'\s+', ' ', sent).strip()

        clean_sentences.append(sent)

    # Gabungkan kembali
    final_text = ' '.join(clean_sentences)
    return final_text

In [13]:
clean_result = advanced_preprocessing(clean_text)

In [14]:
def final_polishing(text):
    """
    TAHAP 3 (FINAL): Menghapus ANGKA dan SEMUA SIMBOL TANDA BACA.
    Hanya menyisakan huruf (a-z, A-Z) dan spasi.
    """


    # 1. KHUSUS APOSTROPHE: Hapus (ganti dengan empty string) agar nyambung
    # Kita handle tanda kutip lurus (') dan kutip miring/smart quote (’) yang sering ada di PDF
    text = re.sub(r"['’]", "", text)
    # 1. Hapus Angka (0-9)
    # Contoh: "VGG19" -> "VGG", "2024" -> ""
    text = re.sub(r'\d+', '', text)

    # 2. Hapus semua simbol & tanda baca (selain huruf dan spasi)
    # Regex [^a-zA-Z\s] berarti: "Hapus karakter apa pun KECUALI huruf dan spasi"
    # Kita ganti dengan spasi ' ' agar kata tidak menempel (word1/word2 -> word1 word2)
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)

    # 3. Normalisasi Spasi (hapus double space akibat penghapusan simbol)
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [15]:
final_cleaned_text = final_polishing(clean_result)

### Speechecker
memperbaiki kata yang typo atau terpisah dengan spasi akibat cleaning

In [18]:
def smart_word_merge(text):
    """
    Menggabungkan kata yang terputus (misal: 'classi cation' -> 'classification')
    dengan mengecek validitas kata gabungan di kamus bahasa Inggris.
    """
    spell = SpellChecker()

    # 1. Tokenisasi sederhana (pisahkan berdasarkan spasi)
    words = text.split()
    merged_words = []
    skip_next = False

    for i in range(len(words)):
        if skip_next:
            skip_next = False
            continue

        current_word = words[i]

        # Cek apakah ini kata terakhir
        if i + 1 < len(words):
            next_word = words[i+1]

            # Bersihkan tanda baca untuk pengecekan kamus
            # Kita hanya ingin cek inti katanya (misal: "classi" dan "fication.")
            curr_clean = re.sub(r'[^a-zA-Z]', '', current_word)
            next_clean = re.sub(r'[^a-zA-Z]', '', next_word)

            combined_clean = curr_clean + next_clean

            # --- LOGIKA UTAMA ---
            # Jika kata gabungan (combined) ADA di kamus (known)
            # DAN kata saat ini (current) TIDAK ADA di kamus (unknown/typo)
            # Maka kemungkinan besar itu adalah kata yang putus.

            if (combined_clean in spell.known([combined_clean])) and \
               (curr_clean not in spell.known([curr_clean])):

                # Gabungkan!
                # Kita pakai combined_clean agar bersih, atau gabung raw stringnya
                # Disini kita gabung raw string tapi hapus spasi diantaranya
                print(f"[AUTO-FIX] Menggabungkan: '{current_word}' + '{next_word}' -> '{combined_clean}'")
                merged_words.append(combined_clean)
                skip_next = True # Karena kata berikutnya sudah kita makan
                continue

        merged_words.append(current_word)

    return " ".join(merged_words)

In [19]:
pengecekan_kata = smart_word_merge(final_cleaned_text)

[AUTO-FIX] Menggabungkan: 'commun' + 'ities' -> 'communities'
[AUTO-FIX] Menggabungkan: 'rapi' + 'd' -> 'rapid'
[AUTO-FIX] Menggabungkan: 'diagnosi' + 'ng' -> 'diagnosing'
[AUTO-FIX] Menggabungkan: 'brai' + 'n' -> 'brain'
[AUTO-FIX] Menggabungkan: 'probabilis' + 'tic' -> 'probabilistic'
[AUTO-FIX] Menggabungkan: 'learni' + 'ng' -> 'learning'
[AUTO-FIX] Menggabungkan: 'proba' + 'bilities' -> 'probabilities'
[AUTO-FIX] Menggabungkan: 'indica' + 'tes' -> 'indicates'


### Preprocessing Stopworld

In [26]:
from nltk.corpus import stopwords
def remove_stopwords(text):
    """
    TAHAP TERAKHIR: Menghapus Stopwords (kata umum tidak bermakna).
    """
    # 1. Ambil daftar stopwords bahasa Inggris
    stop_words = set(stopwords.words('english'))

    # Tambahkan stopword custom spesifik jurnal jika perlu
    custom_stops = {'et', 'al', 'study', 'based', 'using', 'proposed', 'method', 'paper'}
    stop_words.update(custom_stops)

    # 2. Tokenisasi kata
    words = text.split()

    # 3. Filter
    filtered_words = []
    for w in words:
        # Ubah ke lowercase untuk pengecekan stopword
        w_lower = w.lower()

        # Syarat masuk:
        # - Bukan stopword
        # - Panjang kata > 2 huruf (menghindari sisa-sisa 'x', 'y', 'de')
        if w_lower not in stop_words and len(w_lower) > 2:
            filtered_words.append(w) # Simpan kata aslinya (boleh w atau w_lower)

    return " ".join(filtered_words)


In [27]:
stopwords = remove_stopwords(pengecekan_kata)

output_final= "CleaningFInal.txt"
with open(output_final, "w", encoding="utf-8") as f:
    f.write(stopwords)
print(f"[SUKSES] Final cleaned text disimpan ke '{stopwords}'")

[SUKSES] Final cleaned text disimpan ke 'Abstract disease called Alzheimer always put health agencies alarming situation thus necessitating targeted interventions employs VGG convolutional neural network CNN conjunction ADNI dataset enhance precision Alzheimers detection ADNI dataset comprehensive repository neuroimaging data spanning various cognitive states serves foundation robust Alzheimers detection Leveraging VGG CNN architecture renowned image classification capabilities analyze three dimensional MRI scans discern subtle patterns indicative Alzheimer disease fine tuning transfer learning research adapts VGG accurately detect Alzheimers disease several subclasses tries illuminate rapid progression concerned disease Despite challenges Alzheimers complexity high dimensional MRI data ethical considerations findings represent significant advancement Alzheimers disease identification integrating technology medicine research enhances diagnostic accuracy also contributes broader underst

### Preprocessing Final
1. Lemizitation<br>
2. bigram<br>

In [28]:
def advanced_token_processing(text_bersih):
    """
    PIPELINE UTAMA:
    1. Tokenisasi
    2. Lemmatization (networkS -> network)
    3. Bigram Formation (neural + network -> neural_network)
    """
    # 1. TOKENISASI
    tokens = text_bersih.split()

    # 2. LEMMATIZATION (Dilakukan SEBELUM Bigram)
    # Tujuannya agar bentuk jamak (plural) menjadi tunggal dulu
    lemmatizer = WordNetLemmatizer()
    lemma_tokens = []
    for token in tokens:
        # Lemmatize noun (kata benda)
        lemma = lemmatizer.lemmatize(token, pos='n')
        # Lemmatize verb (kata kerja) - opsional tapi bagus untuk jurnal
        lemma = lemmatizer.lemmatize(lemma, pos='v')
        lemma_tokens.append(lemma)

    print(f"[DEBUG] Contoh Lemma: {lemma_tokens[:10]}")

    # 3. BIGRAM FORMATION (Penggabungan Kata)
    # Menggunakan statistik PMI (Pointwise Mutual Information)
    bigram_measures = BigramAssocMeasures()

    # Cari bigram dari token yang SUDAH di-lemma
    finder = BigramCollocationFinder.from_words(lemma_tokens)

    # Filter: Pasangan kata minimal muncul 2 kali (agar tidak menangkap typo)
    finder.apply_freq_filter(2)

    # Ambil 50 Bigram terbaik berdasarkan skor PMI
    top_bigrams = finder.nbest(bigram_measures.pmi, 100)

    # Ubah list bigram menjadi Set agar pencarian cepat
    bigram_set = set(top_bigrams)

    final_tokens = []
    skip_next = False

    for i in range(len(lemma_tokens) - 1):
        if skip_next:
            skip_next = False
            continue

        word1 = lemma_tokens[i]
        word2 = lemma_tokens[i+1]

        # Cek apakah pasangan ini ada di daftar Bigram Top
        if (word1, word2) in bigram_set:
            # GABUNGKAN!
            merged_word = f"{word1}_{word2}"
            final_tokens.append(merged_word)
            skip_next = True
        else:
            # Jika tidak, masukkan sebagai Unigram (kata tunggal)
            final_tokens.append(word1)

    # Handle kata terakhir
    if not skip_next and len(lemma_tokens) > 0:
        final_tokens.append(lemma_tokens[-1])

    return final_tokens

In [30]:
bigram = advanced_token_processing(stopwords)
output_final2 = "hasil_bigram.txt"
with open(output_final2, "w", encoding="utf-8") as f:
    for item in bigram:
        f.write(str(item) + "\n")

[DEBUG] Contoh Lemma: ['Abstract', 'disease', 'call', 'Alzheimer', 'always', 'put', 'health', 'agency', 'alarm', 'situation']


### Build Graph

In [32]:
def build_graph(tokens, window_size=2):
    G = nx.Graph()
    # Tambah semua node
    for t in tokens:
        G.add_node(t)

    # Tambah edge berdasarkan sliding window
    for i in range(len(tokens) - window_size + 1):
        window = tokens[i : i+window_size]
        start = window[0]
        for end in window[1:]:
            if start != end:
                if G.has_edge(start, end):
                    G[start][end]['weight'] += 1
                else:
                    G.add_edge(start, end, weight=1)
    return G

def calculate_all_metrics(G):
    print("Menghitung metrik sentralitas...")
    # 1. PageRank (Relevansi Global)
    pagerank = nx.pagerank(G, weight='weight')

    # 2. Degree Centrality (Popularitas Lokal - banyak koneksi langsung)
    degree = nx.degree_centrality(G)

    # 3. Betweenness Centrality (Jembatan Informasi - penghubung antar kluster)
    betweenness = nx.betweenness_centrality(G)

    # 4. Closeness Centrality (Pusat Penyebaran - mudah dijangkau)
    closeness = nx.closeness_centrality(G)

    return pagerank, degree, betweenness, closeness

# --- 3. VISUALIZATION ---

def visualize_graph(G, pagerank_scores, top_n=40):
    # Ambil Top N node berdasarkan PageRank tertinggi untuk divisualisasikan
    top_nodes = sorted(pagerank_scores.items(), key=lambda x: x[1], reverse=True)[:top_n]
    top_keys = [x[0] for x in top_nodes]

    # Buat Subgraph (agar gambar tidak terlalu ruwet)
    subgraph = G.subgraph(top_keys)

    plt.figure(figsize=(12, 12))

    # Layout algoritma 'spring' agar node menyebar otomatis secara estetis
    pos = nx.spring_layout(subgraph, k=0.5, seed=42)

    # Ukuran node berdasarkan skor PageRank (makin penting makin besar)
    node_sizes = [pagerank_scores[n] * 10000 for n in subgraph.nodes()]

    # Gambar Nodes
    nx.draw_networkx_nodes(subgraph, pos, node_size=node_sizes, node_color='#66c2a5', alpha=0.9)

    # Gambar Edges (Ketebalan berdasarkan bobot co-occurrence)
    weights = [subgraph[u][v]['weight'] for u,v in subgraph.edges()]
    max_w = max(weights) if weights else 1
    widths = [w/max_w * 3 for w in weights]
    nx.draw_networkx_edges(subgraph, pos, width=widths, alpha=0.3, edge_color='gray')

    # Gambar Label
    nx.draw_networkx_labels(subgraph, pos, font_size=10, font_family="sans-serif", font_weight='bold')

    plt.title(f"Visualisasi Graph - Top {top_n} Keywords", fontsize=15)
    plt.axis('off')

    # Simpan ke file
    filename = "keyword_graph.png"
    plt.savefig(filename, format="PNG", dpi=300, bbox_inches='tight')
    plt.close()
    return filename

In [33]:
print(f"2. Membangun Graph dari {len(bigram)} token...")
G = build_graph(bigram, window_size=3)
# Hitung Metrik
pr, deg, bet, clo = calculate_all_metrics(G)

# Fungsi Helper untuk print top 10
def print_top(title, metric_dict):
    print(f"\n--- {title} ---")
    sorted_items = sorted(metric_dict.items(), key=lambda x: x[1], reverse=True)[:10]
    for w, s in sorted_items:
        print(f"   - {w.replace('_', ' ')}: {s:.4f}")

# Tampilkan Hasil Teks
print_top("PageRank (Keyword Utama)", pr)
print_top("Degree Centrality (Paling Terhubung)", deg)
print_top("Betweenness Centrality (Jembatan Topik)", bet)
print_top("Closeness Centrality (Pusat Informasi)", clo)

# Tampilkan Graph
print("\n3. Membuat Visualisasi Graph...")
jumlah_node=len(bigram)
# img_path = visualize_graph(G, pr, top_n=jumlah_node)
img_path = visualize_graph(G, pr, top_n=20)
print(f"   [SUKSES] Gambar disimpan ke: {img_path}")

2. Membangun Graph dari 1433 token...
Menghitung metrik sentralitas...

--- PageRank (Keyword Utama) ---
   - disease: 0.0182
   - Alzheimers: 0.0160
   - VGG: 0.0116
   - use: 0.0113
   - model: 0.0101
   - layer: 0.0075
   - data: 0.0068
   - Fig: 0.0059
   - train: 0.0053
   - image: 0.0052

--- Degree Centrality (Paling Terhubung) ---
   - disease: 0.1270
   - Alzheimers: 0.1085
   - VGG: 0.0913
   - use: 0.0820
   - model: 0.0714
   - layer: 0.0556
   - data: 0.0503
   - Fig: 0.0489
   - ADNI dataset: 0.0410
   - image: 0.0410

--- Betweenness Centrality (Jembatan Topik) ---
   - disease: 0.2000
   - Alzheimers: 0.1559
   - VGG: 0.1270
   - use: 0.1036
   - model: 0.0870
   - Fig: 0.0497
   - data: 0.0467
   - layer: 0.0403
   - ADNI dataset: 0.0370
   - stage: 0.0352

--- Closeness Centrality (Pusat Informasi) ---
   - disease: 0.4370
   - Alzheimers: 0.4288
   - VGG: 0.4257
   - model: 0.4104
   - use: 0.3838
   - Fig: 0.3812
   - data: 0.3776
   - different: 0.3748
   - take: 0

In [34]:
def print_top_keywords(metrics_dict, title, n=20):
    print(f"\n--- {title} (Top {n}) ---")
    # Urutkan dari nilai terbesar ke terkecil
    sorted_keywords = sorted(metrics_dict.items(), key=lambda x: x[1], reverse=True)[:n]

    for i, (word, score) in enumerate(sorted_keywords, 1):
        # Ganti underscore dengan spasi agar bigram terbaca enak (misal: neural_network -> neural network)
        readable_word = word.replace('_', ' ')
        print(f"{i}. {readable_word} ({score:.4f})")

# Panggil fungsi
print_top_keywords(pr, "PageRank Keywords", 20)
print_top_keywords(deg, "Degree Centrality Keywords", 20)


--- PageRank Keywords (Top 20) ---
1. disease (0.0182)
2. Alzheimers (0.0160)
3. VGG (0.0116)
4. use (0.0113)
5. model (0.0101)
6. layer (0.0075)
7. data (0.0068)
8. Fig (0.0059)
9. train (0.0053)
10. image (0.0052)
11. stage (0.0050)
12. ADNI dataset (0.0050)
13. patient (0.0049)
14. detection (0.0046)
15. take (0.0046)
16. dataset (0.0043)
17. output (0.0041)
18. Disease (0.0039)
19. feature (0.0039)
20. filter (0.0038)

--- Degree Centrality Keywords (Top 20) ---
1. disease (0.1270)
2. Alzheimers (0.1085)
3. VGG (0.0913)
4. use (0.0820)
5. model (0.0714)
6. layer (0.0556)
7. data (0.0503)
8. Fig (0.0489)
9. ADNI dataset (0.0410)
10. image (0.0410)
11. train (0.0397)
12. patient (0.0384)
13. take (0.0384)
14. stage (0.0370)
15. feature (0.0357)
16. dataset (0.0344)
17. detection (0.0331)
18. accuracy (0.0317)
19. datasets (0.0317)
20. class (0.0317)


## Opsional jika ingin menampilkan semua node

In [35]:
def visualize_full_graph(G, pagerank_scores, filename="full_graph_viz2.png"):
    """
    Visualisasi seluruh node dengan strategi 'Sparse Labels' dan 'Heatmap Coloring'
    agar tetap terbaca meskipun datanya padat.
    """
    print(f"Sedang merender visualisasi untuk {len(G.nodes())} node... (Mungkin butuh waktu)")

    plt.figure(figsize=(20, 20)) # Ukuran kanvas ekstra besar

    # 1. Layout yang Menyebar (k=0.15 agar tidak terlalu padat)
    pos = nx.spring_layout(G, k=0.15, iterations=50, seed=42)

    # 2. Persiapan Atribut Visual
    # Urutkan node berdasarkan PageRank untuk pewarnaan dan labeling
    sorted_nodes = sorted(pagerank_scores.items(), key=lambda x: x[1], reverse=True)
    top_100_nodes = set([n for n, s in sorted_nodes[:10]]) # Hanya labeli 100 teratas

    # List warna dan ukuran sesuai urutan G.nodes()
    node_sizes = [pagerank_scores[n] * 8000 + 20 for n in G.nodes()] # +20 agar node kecil tetap kelihatan
    node_colors = [pagerank_scores[n] for n in G.nodes()]

    # 3. Gambar Nodes (Pakai colormap 'viridis' atau 'plasma')
    nodes = nx.draw_networkx_nodes(G, pos,
                                   node_size=node_sizes,
                                   node_color=node_colors,
                                   cmap=plt.cm.plasma,
                                   alpha=0.8)

    # 4. Gambar Edges (Tipis dan Transparan)
    # Hanya gambar edge jika bobotnya cukup berarti untuk mengurangi kekusutan
    edges_to_draw = [(u, v) for u, v, d in G.edges(data=True) if d['weight'] > 0]
    weights = [G[u][v]['weight'] for u, v in edges_to_draw]
    max_w = max(weights) if weights else 1
    # Normalisasi ketebalan: garis tipis sekali (0.2) sampai agak tebal (2.0)
    widths = [(w / max_w * 30) + 0.1 for w in weights]

    nx.draw_networkx_edges(G, pos,
                           edgelist=edges_to_draw,
                           width=widths,
                           alpha=0.15,  # Sangat transparan
                           edge_color='#AAAAAA') # Warna abu-abu netral

    # 5. Labeling Selektif (Hanya Top 100)
    labels = {n: n for n in top_100_nodes}
    nx.draw_networkx_labels(G, pos, labels=labels, font_size=8, font_family="sans-serif", font_weight="bold")

    # Tambahkan Colorbar sebagai legenda pentingnya node
    plt.colorbar(nodes, label="PageRank Score")

    plt.title(f"Full Keyword Co-occurrence Graph ({len(G.nodes())} Nodes)\nTop 100 Labels Shown", fontsize=18)
    plt.axis('off')

    plt.savefig(filename, format="PNG", dpi=300, bbox_inches='tight')
    plt.close()
    return filename
img_path = visualize_full_graph(G, pr)

Sedang merender visualisasi untuk 757 node... (Mungkin butuh waktu)
